Have x be an input tuple that includes all input variables of the credit scoring system packaged up

List of all the columns/features still included in objective function model:
- AccountsAccountCategory
- AccountsAccountRefDay
- AccountsAccountRefMonth
- CompanyCategory
- CompanyNameCountNum
- CompanyNameCountX
- CompanyNameLen
- CompanyNameWordLen
- Field1014
- Field1129
- Field1522
- Field1631
- Field17
- Field1865
- Field1871
- Field1885
- Field1977
- Field2267
- Field2298
- Field2304
- Field2316
- Field2447
- Field2483
- Field2497
- Field2502
- Field2506
- Field2616
- Field2619
- Field2705
- Field2815
- Field2816
- Field282
- Field2823
- Field306
- Field448
- Field465
- Field474
- Field477
- Field487
- Field489
- Field541
- Field69
- Field70
- Field972
- FilingId
- MortgagesNumMortCharges
- MortgagesNumMortOutstanding
- MortgagesNumMortPartSatisfied
- MortgagesNumMortSatisfied
- RegAddressCountry
- SIC1
- SIC2
- SIC3
- country
- cty
- dAccountsLastMadeUpDate
- dAccountsNextDueDate
- dConfStmtLastMadeUpDate
- dConfStmtNextDueDate
- dContextInstant
- dIncorporationDate
- dPreviousName_1CONDATE
- dPreviousName_2CONDATE
- dReturnsLastMadeUpDate
- dReturnsNextDueDate
- eAccountsAccountCategory
- eCompanyCategory
- hasF1014
- hasF1129
- hasF1522
- hasF1631
- hasF17
- hasF1865
- hasF1871
- hasF1885
- hasF1977
- hasF2298
- hasF2304
- hasF2316
- hasF2447
- hasF2483
- hasF2497
- hasF2502
- hasF2506
- hasF2616
- hasF2619
- hasF2705
- hasF2815
- hasF282
- hasF306
- hasF448
- hasF465
- hasF474
- hasF487
- hasF489
- hasF541
- hasF69
- hasF70
- hasGNotice
- imdu
- lat
- long
- nSIC
- namechanged
- namechanged2
- oac1
- oac11
- oac2
- oseast1m
- osnrth1m
- ru11ind
- dAccountsTimeGap
- dConfStmtTimeGap
- dReturnsTimeGap
- OtherCompInPcd



_____________________________________________________________________________________________________________


_____________________________________________________________________________________________________________


_____________________________________________________________________________________________________________

NEW PAPER APPROACH:







Let F be the target neural network. We assume that F : Rd → [0, 1]K is a K -class image classifier that takes normalized inputs: each dimension of an input x ∈ Rd represents a single pixel and is bounded between 0 and 1, y ∈ {1,···K} denotes the original label, and the corresponding output F(x) is a K-dimensional vector representing a probability distribution over classes.
- Here K=2 since the only class is "credit-accepted" and "credit-denied"
- d is the amount of input variables for the RF model ?



We assume that, given a correctly classified input image x, the goal of the attacker is to find a perturbation δ such that x + δ is misclassified, i.e., arg maxk F (x + δ) ̸= arg maxk F(x). 
- we mainly want this so that our bad credit score is accpeted and our competitors good credit scores are rejected (but focuse on us being accepted)


We operate in the score-based black-box setting, where we have no knowledge of the internal workings of the network, and a query to the network F yields the entire corresponding K-dimensional output.
- the output here would be the value of "is_failed""

To enforce the notion that the adversarial perturbation should be small, we take the common approach of requiring that ∥δ∥p be smaller than a given threshold ε in some lp norm, where ε varies depending on the classifier. This work considers the l∞ norm, but our attack can easily be adapted to other norms. Finally, we denote the query budget with t; if an adversarial example is not found after t queries to the target network, the attack fails.

Bayesian Optimization consists of two main components: a Bayesian statistical model and an acquisition function. The Bayesian statistical model, also referred to as the surro- gate model, is used for approximating the objective function: it provides a Bayesian posterior probability distribution that describes potential values for the objective function at any candidate point. This posterior distribution is updated each time we query the objective function at a new point. The most common surro- gate model for Bayesian optimization are Gaussian processes (GPs)

we do not need to learn the adversarial perturbation conforming to the actual dimensions of the image. Instead, we learn the perturbation in a much lower dimension. We obtain our final adversarial perturbation by interpolating the learned, low-dimension perturbation to the original input dimension.

Run tests on examples that were first classified as denied credit to see if they are now accepted for credit 

Initially remove contraint ∥δ∥p ≤ε 

In [1]:
# different imports
import pandas as pd
import numpy as np
import pickle
import os

from math import log
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from warnings import catch_warnings
from warnings import simplefilter
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import Matern



In [2]:
# Import data that has already been preprocessed fro RF model
data_dir = os.getcwd()
with open(os.path.join(data_dir, "processing", r"train_preproc.p"), 'rb') as data_file:
    train_data = pickle.load(data_file)
X_train, y_train = train_data[0], train_data[1]

with open(os.path.join(data_dir, "processing", r"test_preproc.p"), 'rb') as data_file:
    test_data = pickle.load(data_file)
X_test, y_test = test_data[0], test_data[1]



In [3]:
#print(X_test.head())
i=0
for x in X_test.columns:
    print(X_test[x].dtype)
    print(X_test[x].name)
    i+=1

print(i)


int64
AccountsAccountCategory
float64
AccountsAccountRefDay
float64
AccountsAccountRefMonth
int64
CompanyCategory
int64
CompanyNameCountNum
int64
CompanyNameCountX
int64
CompanyNameLen
int64
CompanyNameWordLen
float64
Field1014
float64
Field1129
float64
Field1522
float64
Field1631
float64
Field17
float64
Field1865
float64
Field1871
float64
Field1885
float64
Field1977
int64
Field2267
float64
Field2298
float64
Field2304
float64
Field2316
float64
Field2447
float64
Field2483
float64
Field2497
float64
Field2502
float64
Field2506
float64
Field2616
float64
Field2619
float64
Field2705
float64
Field2815
int64
Field2816
float64
Field282
int64
Field2823
float64
Field306
float64
Field448
float64
Field465
float64
Field474
float64
Field477
float64
Field487
float64
Field489
float64
Field541
float64
Field69
float64
Field70
int64
Field972
float64
FilingId
int64
MortgagesNumMortCharges
int64
MortgagesNumMortOutstanding
int64
MortgagesNumMortPartSatisfied
int64
MortgagesNumMortSatisfied
int64
RegAddressC

In [4]:
#Training the Algorithm
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=7, n_estimators=20) # parameters to be adjusted to make most optimal 

In [5]:
classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, n_estimators=20)

In [6]:
# ask oscar what happend with entries that aren't given in model ?--> does the model fault or take this into account

# k in this formular is an iterator over the "other classes" in the output verctor K, but in our 
#example we only have two classes: accepted and denied


# objective function--> underlying "black box" model we are trying to investigate 
def objective(x0, y0, delta):
    
    sumx = x_add(x0, delta)#not sure why the delta_model is of type str here?????
    
    if y0==0: #original input was accepted since is_failed = 0
        return (log(classifier.predict(sumx),1)-log(classifier.predict(sumx),0))
    
    else: #original input was not accepted since is_failed = 1
        return (log(classifier.predict(sumx),0)-log(classifier.predict(sumx),1))

    
# x0 and delta_model should have same length
def x_add(x0, delta):
    result = [None] * len(x0)
    for i in range(0,len(x0)):
        result[i]=(x0.iloc[:, [i]] + delta.iloc[:, [i]])
    return result
        
#ask what the form of the input takes on (oscar should know fomr the preprosseing)

# surrogate or approximation for the objective function
def surrogate(gp_model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")
        return gp_model.predict(X, return_std=True)
 
# probability of improvement acquisition function--> in paper they used expectation of improvement
def acquisition(X, Xsamples, gp_model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(gp_model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(gp_model, Xsamples)
    mu = mu[:, 0]
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs
 
# optimize the acquisition function
def opt_acquisition(X, y, gp_model):
    # random search, generate random samples--> would have to change to my variables instead of only input integers 
    Xsamples = query_points(100)
    # Xsamples = Xsamples.reshape(len(Xsamples), 1) #not sure I need this
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, gp_model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix, 0]


In [7]:
#print(query_points(2))

In [8]:
X = query_points(5) #have to convert to nd array
for x in X:
    print(X)

NameError: name 'query_points' is not defined

In [9]:
def bayes_attack(X0, y0): #X0= initial input values and y0 is inital classification
    # sample the domain sparsely with noise
    n0 = 100
    T = 1000
    
    X = query_points(n0) #have to convert to nd array
    y = asarray([objective(X0, y0, x) for x in X])
    


    # reshape into rows and cols--> not sure what this is 
    X = X.reshape(len(X), 1)
    y = y.reshape(len(y), 1)
    
     # define the model
    gp_model = GaussianProcessClassifier(kernel = Matern) #kernel specified as in Paper
    # fit the model
    gp_model.fit(X, y)
    
    t = n0
    
    # perform the optimization process
    while(t<=T): #increase range to have more accurate result
        
        # select the next point to sample
        delta_t = opt_acquisition(X, y, gp_model)
        
        # sample the point
        v_t= objective(X0, y0, delta_t)
        t = t+1
        
        # summarize the finding
        est, _ = surrogate(gp_model, [[delta_t]])
        print('>x=%.3f, f()=%3f, actual=%.3f' % (delta_t, est, v_t))
        
        if v_t<=0:#might have to change objective function for this to make sense 
            # add the data to the dataset
            X = vstack((X, [[delta_t]]))
            y = vstack((y, [[v_t]]))
            # update the model
            gp_model.fit(X, y)
        else: 
            return (delta_t, "SUCCESSFUL")
    return (delta_t, "UNSUCCESSFUL")

In [10]:
import random

#this functions returns a dataframe of n input sets to query the model with
# we assume n is at least 1
def query_points(n):
    i =1
    points = [new_random_point()]
    while(i<n):
        points.append(new_random_point())
        i=i+1

    names = ['AccountsAccountCategory', 'AccountsAccountRefDay', 'AccountsAccountRefMonth', 'CompanyCategory', 'CompanyNameCountNum', 'CompanyNameCountX',
        'CompanyNameLen', 'CompanyNameWordLen', 'Field1014', 'Field1129', 'Field1522', 'Field1631', 'Field17', 'Field1865', 'Field1871', 'Field1885', 'Field1977',
        'Field2267', 'Field2298', 'Field2304', 'Field2316', 'Field2447', 'Field2483', 'Field2497', 'Field2502', 'Field2506', 'Field2616', 'Field2619', 'Field2705',
        'Field2815', 'Field2816', 'Field282', 'Field2823', 'Field306', 'Field448', 'Field465', 'Field474', 'Field477', 'Field487', 'Field489', 'Field541', 'Field69', 'Field70',
        'Field972', 'FilingId', 'MortgagesNumMortCharges', 'MortgagesNumMortOutstanding', 'MortgagesNumMortPartSatisfied', 'MortgagesNumMortSatisfied', 'RegAddressCountry',
        'SIC1', 'SIC2', 'SIC3', 'country', 'cty', 'dAccountsLastMadeUpDate', 'dAccountsNextDueDate', 'dConfStmtLastMadeUpDate', 'dConfStmtNextDueDate', 'dContextInstant', 
         'dIncorporationDate', 'dPreviousName_1CONDATE', 'dPreviousName_2CONDATE', 'dReturnsLastMadeUpDate', 'dReturnsNextDueDate', 'eAccountsAccountCategory',
        'eCompanyCategory', 'hasF1014', 'hasF1129', 'hasF1522', 'hasF1631', 'hasF17', 'hasF1865', 'hasF1871', 'ÄhasF1885', 'hasF1977', 'hasF2298', 'hasF2304', 'hasF2316', 'hasF2447',
        'hasF2483', 'hasF2497', 'hasF2502', 'hasF2506', 'hasF2616', 'hasF2619', 'hasF2705', 'hasF2815', 'hasF282', 'hasF306', 'hasF448', 'hasF465', 'hasF474', 'hasF487', 'hasF489',
        'hasF541', 'hasF69', 'hasF70', 'hasGNotice', 'imdu', 'lat', 'long', 'nSIC', 'namechanged', 'namechanged2', 'oac1', 'oac11', 'oac2', 'oseast1m', 'osnrth1m', 'ru11ind',
        'dAccountsTimeGap', 'dConfStmtTimeGap', 'dReturnsTimeGap', 'OtherCompInPcd']
    return pd.DataFrame(np.array(points), columns = names)



#generates input values for one input point
def new_random_point():
    return [int64(), float64(), float64(), int64(), int64(), int64(), int64(), int64(), float64(), float64(), float64(), 
            float64(), float64(), float64(), float64(), float64(), float64(), int64(), float64(), float64(), float64(), float64(), float64(), 
            float64(), float64(), float64(), float64(), float64(), float64(), float64(), int64(), float64(), int64(), float64(), float64(),
            float64(), float64(), float64(), float64(), float64(), float64(), float64(), float64(), int64(), float64(), int64(), int64(),
            int64(), int64(), int64(), float64(), float64(), float64(), int64(), int64(), float64(), float64(), float64(), float64(), float64(),
            float64(), float64(), float64(), float64(), float64(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), 
            int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(),
            int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), int32(), 
            rbool(), #i changed rbool here to int32()
            float64(), float64(), float64(), float64(), 
            int32(), int32(), float64(), int64(), int64(), float64(), float64(), int64(), float64(), float64(), float64(), float64()]
    
def int32():
    return random.randrange(-2147483648, 2147483647)

def int64():
    return random.randrange(-9223372036854775808, 9223372036854775807)

def float64():
    return random.random()
    
def rbool():
    p = random.random()
    if(p<0.5): 
        return True
    else: return False
         
#PROBLEM: hasFields are said to have type Int32 but they (and some other fields) only take on values 0 or 1--> need to change 


In [11]:
x0 = query_points(1) # i might have to add column description here? to make it 2D
print(x0)

   AccountsAccountCategory  AccountsAccountRefDay  AccountsAccountRefMonth  \
0            -1.339163e+18               0.374663                 0.722381   

   CompanyCategory  CompanyNameCountNum  CompanyNameCountX  CompanyNameLen  \
0     1.197408e+18         4.281815e+18      -6.799532e+18    6.679560e+18   

   CompanyNameWordLen  Field1014  Field1129  ...      oac1         oac11  \
0       -6.524852e+18   0.957031   0.808746  ...  0.157817 -3.080919e+18   

           oac2  oseast1m  osnrth1m       ru11ind  dAccountsTimeGap  \
0  7.649726e+18  0.345303  0.041396 -3.477685e+18          0.063605   

   dConfStmtTimeGap  dReturnsTimeGap  OtherCompInPcd  
0          0.064193         0.568879        0.922803  

[1 rows x 115 columns]


In [12]:
y0 = classifier.predict(x0)
bayes_attack(x0, y0)

AttributeError: 'str' object has no attribute 'iloc'

In [13]:
#test this bayes opt